<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Advanced Compilation Options with H-Series

A general $SU(4)$ entangler gate is available on the H-Series device. This gate is available as `Optype.TK2` via `tket`. This article showcases the workflow to prepare and submit a circuit with the $SU(4)$ gate with a QVT (Quantum Volume Test) use-case.

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit submitted to hardware doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes. For a listing of the H-Series hardware native gates, see the following links: 
* *System Model H1 Product Data Sheet* available at [System Model H1](https://www.quantinuum.com/hardware/h1); 
* *System Model H2 Product Data Sheet* available at [System Model H2](https://www.quantinuum.com/hardware/h2). 

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

<div style="text-align: center;">
         <img src="figures/hseries-compilation-stack.png" width="800" />
</div>

**Contents:**
* [H-Series Hardware Compilation](#H-Series-Hardware-Compilation)
  * [Compiling to H-Series Hardware Native Gates](#Compiling-to-H-Series-Hardware-Native-Gates)
  * [The General $SU(4)$ Entangler Gate](#The-General-$SU\(4\)$-Entangler-Gate)
  * [Quantinuum API Compiler Options](#Quantinuum-API-Compiler-Options)
* [Use-Case: Quantum Volume Test](#Use-Case:-Quantum-Volume-Test)
  * [Decomposing a random $SU(4)$ unitary into a circuit primitive](#Decomposing-a-random-$SU\(4\)$-unitary-into-a-circuit-primitive)
  * [Building the Quantum Volume Test Circuit](#Building-the-Quantum-Volume-Test-Circuit)
  * [Define Native Two-Qubit Gate to Use With `QuantinuumBackend`](#Define-Native-Two-Qubit-Gate-to-Use-With-QuantinuumBackend)
    * [Generalized $SU(4)$ Entangler](#Generalized-$SU\(4\)$-Entangler)
    * [Arbitrary Angle ZZ Gate](#Arbitrary-Angle-ZZ-Gate)
* [Summary](#Summary)

## H-Series Hardware Compilation

### Compiling to H-Series Hardware Native Gates

On the Quantinuum H-Series devices, there are different native two-qubit gates available. The default native two-qubit gates are an arbitrary-angle two-qubit gate, $Rzz(\theta)$, or a fully entangling two-qubit gate, $ZZ()$. The fully entangling two-qubit gate is equal to, $Rzz(\frac{\pi}{2})$. An additional native gate is available, the General $SU(4)$ Entangler gate, $Rxxyyzz(\alpha, \beta\, \gamma)$. This is available as `OpType.TK2` within `tket`.

`tket` by default, compiles to the arbitrary angle two-qubit gate, $Rzz(\theta)$ gate (`OpType.ZZPhase`). Currently, only one native gate can be specified at a time. This ensures everything aligns in the global operations of the circuit. The other two native gates are `OpType.ZZMax` and `OpType.TK2`.

Usage of the general $SU(4)$ entangler gate requires further specification of the default native two-qubit gate via: 
* `QuantinuumBackendCompilationConfig` option. The `native_2qb_gate` argument needs to be set to `OpType.TK2` to enable use of the SU(4) gate. 
* `QuantinuumBackend.set_compilation_config_target_2qb_gate` instance method. The argument needs to be `OpType.TK2`.


When using `tket`, the following Quantinuum API options are predefined with defaults to prevent further modification of the submitted circuit in the H-Series stack:
* `no-reduce`: `True`
* `no-opt`: True
* `tket-opt-level`: None


<div style="text-align: center;">
         <img src="figures/hseries-stack-options.png" width="800" />
</div>

### The General $SU(4)$ Entangler Gate

The General $SU(4)$ Entangler gate is available in TKET as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). This gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:

$$\small \begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = \exp\left\{-\frac{1}{2} i \alpha (\hat{X} \bigotimes \hat{X}) -\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y}) -\frac{1}{2} i \pi \gamma (\hat{Z} \bigotimes \hat{Z})\right\} \end{equation} \normalsize$$

This gate can be used as follows within TKET.

In [ ]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuit can be converted to a QASM string using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and by specifying the Quantinuum header `hqslib1`.

In [ ]:
from pytket.qasm.qasm import circuit_to_qasm_str

print(circuit_to_qasm_str(circuit, header="hqslib1"))

### Quantum API Compiler Options

The following Quantinuum API options enable circuit compilation in the H-Series stack. They can be defined using the `options` keyword argument in the `QuantinuumBackend` constructor or `process_circuit` (`process_circuits`) instance method.

Quantinuum API Options:
* `no-reduce`: turns off all `tket` optimizations, all hardware compiler gate combination logic, and requires 
    exact 1:1 correspondence of two-qubit gates with gates on the system. This requires the circuit be submitted using 
    one of the native two-qubit gates on the system, otherwise an error will be returned.
* `no-opt`: turns off all `tket` optimizations *and* all hardware compiler gate combination logic.
    * If more than 1 native gate is used in the circuit, the circuit will be rebased to 1 native gate, but no further gate combination logic will occur.
    * The job will fail if `no-opt` is set to `True` and the circuit contains non-native gates. 
* `tket-opt-level`: the `tket` optimization level to apply, with `tket` optimizations turned on, the hardware compiler will provide further gate combination logic as makes sense for ions and transport.
  * `2`: powerful optimizations, compilation can be time-consuming
  * `1`: basic optimization, compiles quickly
  * `0`: rebase the circuit with `tket`
  * `None`: rebase the circuit without `tket`, using the hardware compiler only.

## Use-Case: Quantum Volume Test

Now we illustrate the above compilation options using the Quantum Volume Test. 

Quantum volume is a benchmarking test that was initially proposed by IBM ([arXiv:1811.12926](https://arxiv.org/abs/1811.12926)). It is a test that aims to verify the quality as well as the quantity of qubits on the machine. The test does this by performing rounds of single and two qubit gates between random pairs of qubits for as many rounds as qubits in the test. For example, for quantum volume of $2^N$ where $N=4$, there are 4 layers of repeat operations over 4-qubits. It verifies the quantum computer can perform quality computation with reasonable-sized circuits. The advantage to using quantum volume is that it gives users the confidence that not only do they have the number of qubits to support running their circuit, but the two-qubit gate fidelity meets the threshold to support circuits of significant depth as well. Quantum algorithms need not only qubits, but the ability to run many gates. Quantinuum has steadily been increasing the quantum volume of H-Series machines. 

The workflow involves specifying the use of the desired native gate both in the `tket` compiler options as well as the hardware options when submitting circuits with `process_circuit`.

<div style="text-align: center;">
         <img src="figures/native-gates-workflow.png" width="300" />
</div>

First we import the functions we need in `tket`.   

In [ ]:
import numpy as np

from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.quantinuum.backends.quantinuum import (
    QuantinuumBackendCompilationConfig,
)

### Decomposing a random $SU(4)$ unitary into a circuit primitive

To set up the Quantum Volume test, we start by building up the repeated circuit elements. The function in the code cell below defines a `tket` Circuit Box with [`pytket.circuit.CircBox`](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.CircBox). Circuit Boxes are useful for composing larger circuits from smaller subcircuits that utilize the same set of gates.

The Circuit box below contains a blueprint for the decomposition of a random generalized $SU(4)$ unitary distributed with the Haar Measure into a circuit primitive over 2-qubits. The implementation is based on [arxiv.0609050](http://arxiv.org/abs/math-ph/0609050). The random $SU(4)$ unitary is generated using [`scipy.stats.unitary_group`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.unitary_group.html#scipy-stats-unitary-group).

In [ ]:
from pytket.circuit import CircBox, Circuit, Unitary2qBox
import numpy as np
from scipy.stats import unitary_group


def haar_random_su4_box() -> CircBox:
    r = unitary_group.rvs(dim=4)
    circuit = Circuit(2)
    box = Unitary2qBox(r)
    circuit.add_unitary2qbox(box, 0, 1)
    return CircBox(circuit)

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

circuit = haar_random_su4_box().get_circuit()
render_circuit_jupyter(circuit)

In the code-cell below, the CircBox is inspected and further optimisation are applied on the gate. `DecomposeBoxes` simplifies `OpType.CircBox` operations into the underlying gate operations recursively. Further decompositions  are performed with the `KAKDecomposition` function. We specify use of `OpType.TK2` gate in this function call as the target 2-qubit gate after decomposition. 

The `pytket` sequence pass, consists of two passes:
* [`pytket.passes.DecomposeBoxes`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.DecomposeBoxes)
* [`pytket.passes.KAKDecomposition`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.KAKDecomposition)

Both passes are arguments to [`pytket.passes.SequencePass`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.SequencePass). `SequencePass` allows both passes to be applied to the input circuit with one `apply` call.

In [ ]:
from pytket.passes import DecomposeBoxes, KAKDecomposition, SequencePass
from pytket.circuit import OpType

sequence_pass = SequencePass(
    [DecomposeBoxes(), KAKDecomposition(target_2qb_gate=OpType.TK2)]
)

In [ ]:
sequence_pass.apply(circuit)
render_circuit_jupyter(circuit)

### Building the Quantum Volume Test Circuit

Now we're ready to set up a Quantum Volume test circuit using the Circuit Box we created. The steps to set up the circuit are straightforward using the `CircBox` instances to build up the full circuit.

In [ ]:
circuit = Circuit(4)
for _ in range(4):
    permutation = np.random.permutation(circuit.qubits)
    for i in range(0, 4, 2):
        box = haar_random_su4_box()
        circuit.add_circbox(box, [permutation[i], permutation[i + 1]])
circuit.measure_all()

The `SequencePass` defined in the code cell in the previous subsection is used below on the QVT circuit and we can see the full set of gates in the circuit as well as the result of optimizations performed by `KAKDecomposition`. 

In [ ]:
sequence_pass.apply(circuit)
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

### Define Native Two-Qubit Gate to Use With `QuantinuumBackend`

Now that the circuit has been set up, we will demonstrate submission of the circuit for H-Series backends using both the $SU(4)$ (`OpType.TK2`) gate and $Rzz$ (`OpType.ZZPhase`) as the target two-qubit gate.

#### Generalized $SU(4)$ Entangler

To run a circuit containing the generalized $SU(4)$ gate on H-Series devices, the compilation configuration for the `QuantinuumBackend` needs to have `OpType.TK2` set as the native two-qubit to use by default. This is done using `QuantinuumBackendCompilationConfig` and setting the compilation configuration when intializing `QuantinuumBackend`. By setting `target_2qb_gate`, local tket compilation and submission (for costing and processing) to H-Series with `OpType.TK2` is enabled.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.quantinuum.backends.quantinuum import (
    QuantinuumBackendCompilationConfig,
)

from pytket.circuit import OpType

compilation_config = QuantinuumBackendCompilationConfig(
    allow_implicit_swaps=True, target_2qb_gate=OpType.TK2
)

quantinuum_backend = QuantinuumBackend(
    device_name="H1-1E", compilation_config=compilation_config
)
quantinuum_backend.login()

In [ ]:
compilation_config.target_2qb_gate

In [ ]:
quantinuum_backend.backend_info.gate_set

In [ ]:
qv_circuit_wSU4 = quantinuum_backend.get_compiled_circuit(circuit, optimisation_level=2)
qv_circuit_wSU4.name = "QV Example with SU(4)"
print(f"Number of SU(4) Gates: {qv_circuit_wSU4.n_2qb_gates()}")
render_circuit_jupyter(qv_circuit_wSU4)

In [ ]:
cost = quantinuum_backend.cost(qv_circuit_wSU4, n_shots=100, syntax_checker="H1-1SC")
print(f"SU(4) circuit cost: {cost} HQC")

In [ ]:
handle = quantinuum_backend.process_circuit(qv_circuit_wSU4, n_shots=100)

In [ ]:
result = quantinuum_backend.get_result(handle)
print(result.get_distribution())

In [ ]:
handle = quantinuum_backend.process_circuit(circuit, n_shots=100)

In [ ]:
result = quantinuum_backend.get_result(handle)
print(result.get_distribution())

#### Arbitrary Angle ZZ Gate

The default native two-qubit gate can also be changed using [`set_compilation_config_target_2qb_gate`](https://tket.quantinuum.com/extensions/pytket-quantinuum/api.html#pytket.extensions.quantinuum.QuantinuumBackend.set_compilation_config_target_2qb_gate). In the example below, we change to using a different native two-qubit gate, `OpType.ZZPhase` and display the compiled circuit after `tket` compilation.

In [ ]:
quantinuum_backend.set_compilation_config_target_2qb_gate(OpType.ZZPhase)

In [ ]:
qv_circuit_no_SU4 = quantinuum_backend.get_compiled_circuit(
    circuit, optimisation_level=2
)
qv_circuit_no_SU4.name = "QV Example without SU(4)"
print(f"Number of ZZPhase Gates: {qv_circuit_no_SU4.n_2qb_gates()}")
render_circuit_jupyter(qv_circuit_no_SU4)

### Summary

`pytket-quantinuum` enables you to change the default native two-qubit gate for H-Series devices and emulators. This can be acheived with the `Quantinuum` API compiler options or with `QuantinuumBackendCompilationConfig` and `QuantinuumBackend.set_compilation_config_target_2qb_gate`.

The Quantum Volume Test is one use case where the $SU(4)$ gate (`OpType.TK2`) can be employed. `QuantinuumBackend` can be used to compile circuits to a gatset that uses the $SU(4)$ gate as well as to submit those circuits for processing and costing via the H-Series service.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>